In [1]:
import fglib
from fglib import graphs,nodes,rv, inference
import pyldpc
import numpy as np
from random import uniform

# a) Construct factor graph based on some H

In [2]:
# Generate H
# Reference: https://github.com/hichamjanati/pyldpc-tutos/blob/master/Tutorials/pyLDPC-Tutorial-Basics.ipynb

N = 128  # Number of bits
dim_codeword = 2*N
d_v = 1 # Number of ones per column, must be lower than d_c (because H must have more rows than columns)
d_c = 2 # Number of ones per row, must divide n (because if H has m rows: m*d_c = n*d_v (compute number of ones in H))

H = pyldpc.RegularH(dim_codeword,d_v,d_c)
print("Regular parity-check matrix H({},{},{}):\n\n".format(dim_codeword,d_v,d_c),H)

Regular parity-check matrix H(256,1,2):

 [[1 1 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 1 1]]


In [30]:
def construct_check_factor(check_fnode):
    nodes_in_check = list(check_fnode.neighbors())
    n_nodes_in_check = len(nodes_in_check)
    pmf_shape = [2]*n_nodes_in_check
    pmf_out = np.zeros(pmf_shape)
    all_idx = np.where(pmf_out==0)
    pmf_values = 1-np.stack(all_idx).sum(0)%2
    pmf_out[all_idx] = pmf_values
    return rv.Discrete(pmf_out, *nodes_in_check)

In [31]:
def construct_prior_factor(prior_fnode, observed_r, error_rate):
    if observed_r==0:
        pmf = np.array([1-error_rate, error_rate])
    else:
        pmf = np.array([error_rate, 1-error_rate])
    return  rv.Discrete(pmf, prior_fnode)

In [32]:
# Construct factor graph
def construct_graph_with_H_and_observations(H, observations, error_rate=0.05):
    n_checks, n_bits = H.shape
    # Construct empty factor graph
    factor_graph = graphs.FactorGraph()
    # Construct bits V nodes
    bits_vnodes = [nodes.VNode('x%d'%n) for n in range(n_bits)]
    # Construct checks F nodes
    checks_fnodes = [nodes.FNode('h%d'%m) for m in range(n_checks)]
    # Construct priors F nodes
    priors_fnodes = [nodes.FNode('f%d'%m) for m in range(n_bits)]
    
    # Register Nodes to the graph
    factor_graph.set_nodes(bits_vnodes)
    factor_graph.set_nodes(checks_fnodes)
    factor_graph.set_nodes(priors_fnodes)
    
    # Construct Edges between priors and bits
    factor_graph.set_edges(zip(priors_fnodes, bits_vnodes))
    # Constrct Edges between bits and checks
    check_idx, bit_idx = np.where(H==1)
    for m, n in zip(check_idx, bit_idx):
        factor_graph.set_edge(checks_fnodes[m], bits_vnodes[n])
    
    # Initialize check factors
    for check_fnode in checks_fnodes:
        check_fnode.factor = construct_check_factor(check_fnode)
    # Initialize prior factors
    for prior_fnode, observed_r in zip(priors_fnodes, observations):
        prior_fnode.factor = construct_prior_factor(prior_fnode,observed_r,error_rate)
    return factor_graph

In [33]:
# Testing cell
fg = construct_graph_with_H(H)

# b) Decoding Test

In [40]:
# Construct Coding Matrix tG
tG = pyldpc.CodingMatrix(H)
G  = tG.T

In [70]:
# Original message: 
message= pyldpc.imagesformat.int2bitarray(19931125,128)

In [92]:
print('Original Messages:')
print(message)
print('Shape:', message.shape)

Original Messages:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1]
Shape: (128,)


In [110]:
# Define binary symmetric channel
def binary_symmetric_channel(message, error_rate):
    error_bits_idx = np.where(np.random.choice([0,1], p=(1-error_rate,error_rate),size=message.shape)==1)[0] 
    corrupted = message
    corrupted[error_bits_idx] = 1-corrupted[error_bits_idx]
    return corrupted

In [111]:
# Encode original message with some error rate
error_rate = 0.05 # symmetric noise error
codeword = binary_symmetric_channel(message, error_rate)